# Preenchimento reativo
Mostra o comportamento da rede utilizando o preenchimento reativo. Essa simulação registra informações das ``requests``:

    - ID da requisição 
    - Alice
    - Bob
    - Fidelidade Mínima
    - Número de EPRs
    - Início (Quando começou a ser tratada)
    - Término (Quando foi atendida)
    - Novo Registro (Se foi necessário criar uma regra para atender, ou se já foi registrado)

In [1]:
# Importando as dependências
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request, clear_file
from quantumnet.objects import time

Informações para a simulação

In [2]:
# Caminho do arquivo de saída
path = "resultados/hibrido/sync.csv"
# Limpa o arquivo de saída
clear_file(path)
# Sobre a rede
rede = Network()
rede.n_initial_eprs = 10
rede.n_initial_qubits = 10
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Sobre o controlador
controlador = Controller(rede)
controlador.default_ttl = 100
# Sobre as requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 100
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))

Funções responsáveis pela passagem de tempo

In [3]:
def update_rules(host, time_for_update):
    """
    Atualiza as regras de um host
    Args:
        host (Host): Host que terá as regras atualizadas
        time_for_update (int): Tempo para atualização das regras
    """
    # Percorre a tabela de fluxo do host
    for match in host.flow_table.copy():
        # Recebe o objeto da regra, somente
        rule = host.flow_table[match][1]
        # Incrementa a idade da regra
        for t in range(time_for_update):
            rule.increment_age()
        # Verifica se a regra expirou
        if rule.ttl <= rule.age:
            if rule.hit_count == 0:
                host.flow_table.pop(match)
                print(f"[Time {time.get_current_time()}] Regra {rule} expirada em {host}")
            else:
                print(f"Regra reutilizada {rule} em {host}")
                rule.hit_count = 0
                rule.age = rule.ttl/2

def update_time(time_for_update):
    """
    Atualiza o tempo e as regras

    Args:
        time_for_update (int): Tempo para incrementar o time-slot.
    """
    # Atualiza o tempo
    for t in range(time_for_update):
        time.increment()
        # Atualiza os recursos de 10 em 10
        if time.get_current_time() % 10 == 0:
            rede.refresh_resources(num_qubits=10, num_eprs=10)
            print(f"[Time {time.get_current_time()}] Recursos atualizados")
    # Atualiza as regras
    for host in rede.hosts:
        update_rules(rede.get_host(host), time_for_update)

Preenchimento reativo, baseado nas requests.

    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.

In [4]:
for request in requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time.get_current_time()
            update_time(3)
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time() 
            # Registra no CSV como um novo registro
            register_request(request, "Novo Registro", path)
        else:  # Caso já exista a regra
            request.starttime = time.get_current_time()
            update_time(1)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time()
            # Registra no CSV como já registrado
            register_request(request, "Já Registrado", path)
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

[Time 0] Processando requisição R-210830...
Decisão aplicada: (High Fidelity,)
Request R-210830: Start Time = 0, End Time = 3
[Time 3] Processando requisição R-12E710...
Decisão aplicada: (High Fidelity,)
Request R-12E710: Start Time = 3, End Time = 6
[Time 6] Processando requisição R-12E850...
Decisão aplicada: (Normal E2E,)
[BasicRule] Passo: 1
[BasicRule] Passo: 2
[BasicRule] Ação: Swap(0-4-8)
Running Swap(0-4-8)
[BasicRule] Ação: Swap(8-9-10)
Running Swap(8-9-10)
[BasicRule] Passo: 3
[BasicRule] Ação: Swap(0-8-10)
Running Swap(0-8-10)
Request R-12E850: Start Time = 6, End Time = 9
[Time 9] Processando requisição R-112D70...
[Time 10] Recursos atualizados
Decisão aplicada: (Normal E2E,)
[BasicRule] Passo: 1
Request R-112D70: Start Time = 9, End Time = 12
[Time 12] Processando requisição R-112EA0...
Decisão aplicada: (High Fidelity,)
Request R-112EA0: Start Time = 12, End Time = 15
[Time 15] Processando requisição R-10B9B0...
Decisão aplicada: (Normal E2E,)
[BasicRule] Passo: 1
[Basi

Registro de recursos

In [5]:
rede.registry_of_resources

{'qubits created': 3480,
 'qubits used': 0,
 'eprs created': 10180,
 'eprs used': 0}